# Face_Detection using [https://realpython.com/face-recognition-with-python/]

In [18]:
import cv2

# for running on colab
# from google.colab.patches import cv2_imshow

from matplotlib import pyplot as plt
from PIL import Image, ImageDraw

import time

from collections import Counter

import pickle
from pathlib import Path
import argparse

import face_recognition

## Step 1: Prepare Your Environment and Data

In [19]:
# mkdir face_recognizer

In [20]:
# cd face_recognizer

In [21]:
# mkdir output

In [22]:
# mkdir training

In [23]:
# mkdir validation

## Step 2: Load Training Data and Train Your Model

HOG (histogram of oriented gradients) is a common technique for object detection. For this tutorial, you only need to remember that it works best with a CPU.
CNN (convolutional neural network) is another technique for object detection. In contrast to a HOG, a CNN works better on a GPU, otherwise known as a video card.

In [24]:
DEFAULT_ENCODINGS_PATH = Path('output/encodings.pkl')
BOUNDING_BOX_COLOR = 'blue'
TEXT_COLOT = 'white'

In [25]:
parser = argparse.ArgumentParser(description = 'Recognize faces in an image')
parser.add_argument('--train', action='store_true', help='train on input data')
parser.add_argument('--validate', action='store_true', help='validate trained model')
parser.add_argument('--test', action='store_true', help='test the model with an unknown image')
parser.add_argument(
    '-m',
    action='store',
    default='hog',
    choices=['hog', 'cnn'],
    help='Which model to use for training: hog (CPU), cnn (GPU)'
)
parser.add_argument('-f', action='store', help='Path to an image with an unknown face')
args = parser.parse_args()

In [26]:
Path('training').mkdir(exist_ok=True)
Path('output').mkdir(exist_ok=True)
Path('validation').mkdir(exist_ok=True)

In [27]:
def encode_known_faces(model: str = 'hog', encodings_location: Path = DEFAULT_ENCODINGS_PATH) -> None:
    names = []
    encodings = []
    
    for filepath in Path('training').glob('*/*'):
        name  = filepath.parent.name
        image = face_recognition.load_image_file(filepath)
        
        # bounding box
        face_locations = face_recognition.face_locations(image, model = model)
        face_encodings = face_recognition.face_encodings(image, face_locations)
        
        for encoding in face_encodings:
            names.append(name)
            encodings.append(encoding)
            
    name_encodings = {'names': names, 'encodings': encodings}
    with encodings_location.open(mode='wb') as f:
        pickle.dump(name_encodings, f)

In [37]:
# encode_known_faces()

In [39]:
# with open('output/encodings.pkl', mode='rb') as f:
#     loaded_encodings = pickle.load(f)
# print(loaded_encodings['names'])

['ben_afflek', 'ben_afflek', 'ben_afflek', 'ben_afflek', 'ben_afflek', 'ben_afflek', 'ben_afflek', 'ben_afflek', 'ben_afflek', 'ben_afflek', 'ben_afflek', 'ben_afflek', 'ben_afflek', 'elon_musk', 'elon_musk', 'elon_musk', 'elon_musk', 'elon_musk', 'elon_musk', 'elon_musk', 'elon_musk', 'elon_musk', 'elon_musk', 'elon_musk', 'elon_musk', 'elon_musk', 'elon_musk', 'elon_musk', 'elon_musk', 'elon_musk', 'elton_john', 'elton_john', 'elton_john', 'elton_john', 'elton_john', 'elton_john', 'elton_john', 'elton_john', 'elton_john', 'elton_john', 'elton_john', 'elton_john', 'elton_john', 'jerry_seinfeld', 'jerry_seinfeld', 'jerry_seinfeld', 'jerry_seinfeld', 'jerry_seinfeld', 'jerry_seinfeld', 'jerry_seinfeld', 'jerry_seinfeld', 'jerry_seinfeld', 'jerry_seinfeld', 'jerry_seinfeld', 'jerry_seinfeld', 'jerry_seinfeld', 'jerry_seinfeld', 'jerry_seinfeld', 'jobiden', 'jobiden', 'jobiden', 'jobiden', 'jobiden', 'jobiden', 'jobiden', 'jobiden', 'jobiden', 'jobiden', 'jobiden', 'jobiden', 'jobiden', '

## Step 3: Recognize Unlabeled Faces

In [29]:
def recognize_faces(
    image_location: str,
    model: str = 'hog',
    encodings_location: Path = DEFAULT_ENCODINGS_PATH
) -> None:
    with encodings_location.open(mode = 'rb') as f:
        loaded_encodings = pickle.load(f)
        
    input_image = face_recognition.load_image_file(image_location)
    
    input_face_locations = face_recognition.face_locations(input_image, model=model)
    input_face_encodings = face_recognition.face_encodings(input_image, input_face_locations)
    
    pillow_image = Image.fromarray(input_image)
    draw = ImageDraw.Draw(pillow_image)
    
    for bounding_box, unknown_encoding in zip(
        input_face_locations, input_face_encodings
    ):
        name = _recognize_face(unknown_encoding, loaded_encodings)
        if not name:
            name = 'Unknown'
        # print(name, bounding_box)
        _display_face(draw, bounding_box, name)
        
    del draw
    pillow_image.show()

In [30]:
def _recognize_face(unknown_encoding, loaded_encodings):
    boolean_matches = face_recognition.compare_faces(
        loaded_encodings['encodings'], unknown_encoding
    )
    votes = Counter(
        name
        for match, name in zip(boolean_matches, loaded_encodings['names'])
    )
    if votes:
        return votes.most_common(1)[0][0]

## Step 4: Display Results

In [31]:
def _display_face(draw, bounding_box, name):
    top, right, bottom, left = bounding_box
    draw.rectangle(((left, top), (right, bottom)), outline = BOUNDING_BOX_COLOR)
    text_left, text_top, text_right, text_bottom = draw.textbbox(
        (left, bottom), name
    )
    draw.rectangle(
        ((text_left, text_top), (text_right, text_bottom)),
        fill='blue',
        outline='blue'
    )
    draw.text((text_left, text_top), name, fill='white')

In [32]:
# test_images
# recognize_faces('amandeep.jpg')

In [33]:
# recognize_faces('BB12XCJF.jpeg')

## Step 5: Validate Your Model

In [34]:
def validate(model: str = 'hog'):
    for filepath in Path('validation').rglob('*'):
        if filepath.is_file():
            recognize_faces(image_location = str(filepath.absolute()), model = model)

In [35]:
# validate()

## Step 6: Add Command-Line Arguments

In [36]:
if __name__ == '__main__':
    if args.train:
        encode_known_faces(args.m)
    if args.validate:
        validate(args.m)
    if args.test:
        recognize_faces(image_location=args.f, model=args.m)
        

## Step 7: Perform Face Recognition With Python